## Chapter 7

### Algorithm Examples

In [1]:
from importlib import reload
import utilities as utils
reload(utils)
import numpy as np
import matplotlib.pyplot as plt

### Problem 1

Identify the class conditional probability densities: $Pr(x = x^* \ | \ w=\text{unripe})$ and $Pr(x = x^* \ | \ w=\text{ripe})$.  The class conditional probability densities should be constructed with the following facts in mind:

* The conditional densities may be multimodal.  Evaluation of the distribution of the data will tell how many bases are required to best model the data
* There may be outliers in one or both of the conditional densities.  Exploratory analysis of the data will indicate if this is the case or not.
* The data $x$ is low-dimensional, so factor analysis is unnecessary.

With these facts in mind, the best choice would be to consider a _robust mixture model_; see Figure 7.3.  The choice and parameterization of student-t densities required to represent the class conditional densities can be constructed using the EM algorithm.  Now, given modeled representations of $Pr(x = x^* \ | \ w=\text{unripe})$ and $Pr(x = x^* \ | \ w=\text{ripe})$, we can use this information to complete our classifier:

\begin{eqnarray}
Pr(w=\text{unripe} \ | \ x = x^*) &=& \frac{Pr(x = x^* \ | \ w=\text{unripe})Pr(w=\text{unripe})}{Pr(x = x^* \ | \ w=\text{unripe})Pr(w=\text{unripe}) + Pr(x = x^* \ | \ w=\text{ripe})Pr(w=\text{ripe})} \\
Pr(w=\text{ripe} \ | \ x = x^*) &=& \frac{Pr(x = x^* \ | \ w=\text{ripe})Pr(w=\text{ripe})}{Pr(x = x^* \ | \ w=\text{unripe})Pr(w=\text{unripe}) + Pr(x = x^* \ | \ w=\text{ripe})Pr(w=\text{ripe})}
\end{eqnarray}

The only remaining consideration is the prior density over the world state.  This can be represented by a Bernoulli distribution with the parameter $\lambda$ chosen to either reflect empirical data (e.g. from a training set) or equal likelihood ($\lambda = 0.5$).

### Problem 2

Misrepresentation of labels in the underlying dataset would appear as outliers in the class conditional densities.  Representation of these densities as student-t distributions could help mitigate the effect of these outliers.

_As an aside: If you, as the data scientist, are aware that there are issues with the underlying data, you should probably retrain your model with the appropriate label corrections applied!_ 

### Problem 3

The density of the hidden variable $q_i(\mathbf{h}_i)$:

\begin{eqnarray}
q_i(\mathbf{h}_i) &=& \frac{\lambda_k \text{Norm}_{\mathbf{x}_i} [\mathbf{\mu}_k, \Sigma_k]}{\sum_{j=1}^K \lambda_j \text{Norm}_{\mathbf{x}_i} [\mathbf{\mu}_j, \Sigma_j]} \\
&\triangleq& r_{ik}
\end{eqnarray}

The log-likelihood, with Lagrange-multiplier for the constraint $\sum_{j=1}^k \lambda_j = 1$, is:
\begin{eqnarray}
L = \sum_{i=1}^I r_{ik} \bigl [ \log \lambda_k - 0.5 (\mathbf{x}_i - \mathbf{\mu}_k)^T \Sigma_k^{-1} (\mathbf{x}_i - \mathbf{\mu}_k) \bigr ]
\end{eqnarray}

_Before differentiation of $L$, note that $r_{ik}$ is constant during the M step of the EM algorithm!_

Take the derivatives and find values that make them vanish:
\begin{eqnarray}
0 &=& \frac{\partial L}{\partial \mathbf{\mu}_k} \\
&=& \sum_{i=1}^I r_{ik} \Sigma_k^{-1} (\mathbf{x}_i - \mathbf{\hat \mu}_k) \\
\sum_{i=1}^I r_{ik} \mathbf{\hat \mu}_k &=& \sum_{i=1}^I r_{ik} \mathbf{x}_i \\
\mathbf{\hat \mu}_k &=& \frac{\sum_{i=1}^I r_{ik} \mathbf{x}_i}{\sum_{i=1}^I r_{ik}} \\
0 &=& \frac{\partial L}{\partial \Sigma_k} \\
\text{see Appendix C.38} &=& \sum_{i=1}^I -0.5 r_{ik} \bigl [\hat \Sigma_k^{-T} - \hat \Sigma_k^{-2} (\mathbf{x}_i - \mathbf{\hat \mu}_k) (\mathbf{x}_i - \mathbf{\hat \mu}_k)^T \bigr ] \\
\hat \Sigma_k^{-1} \sum_{i=1}^I r_{ik} &=& \hat \Sigma_k^{-2} \sum_{i=1}^I r_{ik} (\mathbf{x}_i - \mathbf{\hat \mu}_k) (\mathbf{x}_i - \mathbf{\hat \mu}_k)^T \\
\hat \Sigma_k &=& \frac{\sum_{i=1}^I r_{ik} (\mathbf{x}_i - \mathbf{\hat \mu}_k) (\mathbf{x}_i - \mathbf{\hat \mu}_k)^T}{\sum_{i=1}^I r_{ik}} \\
0 &=& \frac{\partial L}{\partial \lambda_k} \\
&=& \sum_{i=1}^I \frac{r_{ik}}{\hat \lambda_k} + \nu \\
0 &=& \sum_{i=1}^I r_{ik} + \nu \hat \lambda_k \\
\text{(must hold for all $k$)} \ \ \nu \sum_{j=1}^K \hat \lambda_j + \sum_{j=1}^K \sum_{i=1}^I r_{ij} &=& 0 \\
\nu \cdot 1 + \sum_{j=1}^K \sum_{i=1}^I r_{ij} &=& 0 \\
\nu &=& -\sum_{j=1}^K \sum_{i=1}^I r_{ij} \\
0 &=& \sum_{i=1}^I r_{ik} + \nu \hat \lambda_k \\
&=& \sum_{i=1}^I r_{ik} -\sum_{j=1}^K \sum_{i=1}^I r_{ij} \hat \lambda_k \\
\sum_{j=1}^K \sum_{i=1}^I r_{ij} \hat \lambda_k &=& \sum_{i=1}^I r_{ik} \\
\hat \lambda_k &=& \frac{\sum_{i=1}^I r_{ik}}{\sum_{j=1}^K \sum_{i=1}^I r_{ij}}
\end{eqnarray}

The optimal values for the M step are:
\begin{eqnarray}
\mathbf{\hat \mu}_k &=& \frac{\sum_{i=1}^I r_{ik} \mathbf{x}_i}{\sum_{i=1}^I r_{ik}} \\
\hat \Sigma_k &=& \frac{\sum_{i=1}^I r_{ik} (\mathbf{x}_i - \mathbf{\hat \mu}_k) (\mathbf{x}_i - \mathbf{\hat \mu}_k)^T}{\sum_{i=1}^I r_{ik}} \\
\hat \lambda_k &=& \frac{\sum_{i=1}^I r_{ik}}{\sum_{j=1}^K \sum_{i=1}^I r_{ij}}
\end{eqnarray}

### Problem 4
The beta-mixture model would take the form:
\begin{eqnarray}
Pr(\mathbf{x} | \mathbf{\Theta}) &=& \sum_{k=1}^K \lambda_k \text{Beta}_\mathbf{x} [ \bar \alpha_k, \bar \beta_k ]
\end{eqnarray}

The objective is to find the $\hat \lambda_k, \hat \alpha_k, \hat \beta_k$ for $1 \le k \le K$ that best fits the observed $I$ data points $\{ \mathbf{x} \}_i^I$.  The EM algorithm can be used to iteratively approach the best fit $\mathbf{\Theta}$:

##### E-step
Compute the probability that $Pr(\mathbf{h}_i=k \ | \ \mathbf{x}_i, \mathbf{\Theta}^{[t]})$ that the $k$th beta distribution was responsible for datapoint $\mathbf{x}_i$.  Repeat this step for all $k$ and $i$.  This step maximizes the likelihood bound given distributions of hidden variables $\mathbf{h}_i$.

##### M-step
Maximize likelihood by computing the optimal values $\hat \lambda_k^{[t+1]}, \hat \alpha_k^{[t+1]}, \hat \beta_k^{[t+1]}$ given the probabilities computed during the E-step.

At each iteration, check to see if the estimates for $\hat \lambda_k, \hat \alpha_k, \hat \beta_k$ have converged.  If not, repeat the procedure.  If so, record the values and stop iterating.

### Problem 5

\begin{eqnarray}
\text{Stud}_x [\mu, \sigma^2, \nu ] &=& \int_0^\infty \text{Norm}_x \Bigl [ \mu, \frac{\sigma^2}{h} \Bigr ] \text{Gam}_h \Bigl [ \frac{\nu}{2}, \frac{\nu}{2} \Bigr ] dh \\
&=& \int_0^\infty \frac{\sqrt{h}}{\sqrt{2 \pi \sigma^2}} \exp \Bigl [-\frac{h(x-\mu)^2}{2 \sigma^2} \Bigr ] \frac{\bigl( \frac{\nu}{2} \bigr )^{\nu/2}}{\Gamma \bigl [ \frac{\nu}{2} \bigr ]} \exp \Bigl [ -\frac{\nu h}{2} \Bigr ] h^{\nu/2 - 1} dh \\
&=& \frac{1}{\sqrt{2 \pi \sigma^2}} \frac{\bigl( \frac{\nu}{2} \bigr )^{\nu/2}}{\Gamma \bigl [ \frac{\nu}{2} \bigr ]} \int_0^\infty h^{(\nu-1)/2} \exp \Bigl [-\frac{h}{2} \bigl ( \frac{(x-\mu)^2}{\sigma^2} + \nu \bigr ) \Bigr ] dh \\
&\triangleq& \frac{1}{\sqrt{2 \pi \sigma^2}} \frac{\bigl( \frac{\nu}{2} \bigr )^{\nu/2}}{\Gamma \bigl [ \frac{\nu}{2} \bigr ]} \int_0^\infty h^a \exp [-hb] dh \\
a &\triangleq& \frac{\nu-1}{2} \\
b &\triangleq& -\frac{1}{2} \bigl ( \frac{(x-\mu)^2}{\sigma^2} + \nu \bigr )
\end{eqnarray}

The integral above can be evaluated with a table lookup.

\begin{eqnarray}
\frac{1}{\sqrt{2 \pi \sigma^2}} \frac{\bigl( \frac{\nu}{2} \bigr )^{\nu/2}}{\Gamma \bigl [ \frac{\nu}{2} \bigr ]} \int_0^\infty h^a \exp [-hb] dh &=& \frac{1}{\sqrt{2 \pi \sigma^2}} \frac{\bigl( \frac{\nu}{2} \bigr )^{\nu/2}}{\Gamma \bigl [ \frac{\nu}{2} \bigr ]} \frac{\Gamma [b + 1 ]}{a^{b+1}} \\
&=& \frac{\nu^{\nu/2}}{2^{\nu/2} \sqrt{2 \pi \sigma^2}} \frac{\Gamma \bigl [ \frac{\nu+1}{2} \bigr ]}{\Gamma \bigl [ \frac{\nu}{2} \bigr ]} \frac{1}{2^{-(\nu+1)/2}} \bigl ( \frac{(x-\mu)^2}{\sigma^2} + \nu \bigr )^{-(\nu+1)/2} \\
&=& \frac{\nu^{\nu/2}}{2^{(\nu+1)/2} 2^{-(\nu+1)/2} \sqrt{\pi \sigma^2}} \frac{\Gamma \bigl [ \frac{\nu+1}{2} \bigr ]}{\Gamma \bigl [ \frac{\nu}{2} \bigr ]} \frac{1}{\nu^{(\nu+1)/2}} \bigl ( \frac{(x-\mu)^2}{\nu \sigma^2} + 1 \bigr )^{-(\nu+1)/2} \\
&=& \frac{1}{\sqrt{\nu \pi \sigma^2}} \frac{\Gamma \bigl [ \frac{\nu+1}{2} \bigr ]}{\Gamma \bigl [ \frac{\nu}{2} \bigr ]} \bigl ( \frac{(x-\mu)^2}{\nu \sigma^2} + 1 \bigr )^{-(\nu+1)/2} \\
\end{eqnarray}

The final line shows the desired result holds true.

### Problem 6

\begin{eqnarray}
\text{Gam}_z [\alpha, \beta] &=& \frac{\beta^\alpha}{\Gamma[\alpha]} \exp [-\beta z] z^{\alpha - 1}
\end{eqnarray}

The maximum likelihood value for the gamma distribution occurs when $\frac{d \text{Gam}_z [\alpha, \beta]}{dz} = 0$.

\begin{eqnarray}
0 &=& \frac{d \text{Gam}_z [\alpha, \beta]}{dz} \\
&=& \frac{\beta^\alpha}{\Gamma[\alpha]} \exp [-\beta \hat z] {\hat z}^{\alpha - 1} {\hat z}^{\alpha - 2} \bigl [ -\beta \hat z + (\alpha -1) \bigr ] \\
-\beta \hat z + (\alpha -1)&=& 0 \\
\beta \hat z &=& \alpha -1 \\
\hat z &=& \frac{\alpha - 1}{\beta}
\end{eqnarray}

### Problem 7

On the decision boundary,

\begin{eqnarray}
Pr(w = 1 \ | \ x) &=& 0.5
&=& Pr(w = 0 \ | \ x)
\end{eqnarray}

regardless of the prior.  For this problem, use a uniform prior:

\begin{eqnarray}
Pr(w = 0) = Pr(w = 1) = 0.5
\end{eqnarray}

Using Bayes' rule:

\begin{eqnarray}
Pr(w = 1 \ | \ x) &=& \frac{Pr(x \ | \ w = 1) Pr(w=1)}{Pr(x \ | \ w = 0) Pr(w=0) + Pr(x \ | \ w = 1) Pr(w=1)} \\
\text{(plug in values)} \ \ 0.5 &=& \frac{0.5 \cdot Pr(x \ | \ w = 1)}{0.5 \cdot Pr(x \ | \ w = 0) + 0.5 \cdot Pr(x \ | \ w = 1)} \\
0.5 \cdot Pr(x \ | \ w = 0) + 0.5 \cdot Pr(x \ | \ w = 1) &=& Pr(x \ | \ w = 1) \\
0.5 \cdot Pr(x \ | \ w = 0) &=& 0.5 \cdot Pr(x \ | \ w = 1) \\
Pr(x \ | \ w = 0) &=& Pr(x \ | \ w = 1)
\end{eqnarray}

Model the probability distributions as univariate normals:

\begin{eqnarray}
Pr(x \ | \ w = 0) &=& \text{Norm}_x [ \mu_0, \sigma^2_0 ] \\
Pr(x \ | \ w = 1) &=& \text{Norm}_x [ \mu_1, \sigma^2_1 ]
\end{eqnarray}

So, for a point $x^*$ on the decision boundary, the condition above can be rewritten as:

\begin{eqnarray}
Pr(x^* \ | \ w = 0) &=& Pr(x^* \ | \ w = 1) \\
\text{Norm}_{x^*} [ \mu_0, \sigma^2_0 ] &=& \text{Norm}_{x^*} [ \mu_1, \sigma^2_1 ] \\
\log \text{Norm}_{x^*} [ \mu_0, \sigma^2_0 ] &=& \log \text{Norm}_{x^*} [ \mu_1, \sigma^2_1 ] \\
-\frac{I}{2} \log \sigma_0^2 - \frac{I}{2} \log 2 \pi - \frac{(x^* - \mu_0)^2}{2 \sigma_0^2} &=& -\frac{I}{2} \log \sigma_1^2 - \frac{I}{2} \log 2 \pi - \frac{(x^* - \mu_1)^2}{2 \sigma_1^2} \\
\frac{(x^* - \mu_1)^2}{2 \sigma_1^2} - \frac{(x^* - \mu_0)^2}{2 \sigma_0^2} + \frac{I}{2} \bigl ( \log \sigma_0^2 - \log \sigma_1^2 \bigr ) &=& 0
\end{eqnarray}

which can be rewritten in the desired form of a quadratic equation:

\begin{eqnarray}
\Bigl ( \frac{1}{2 \sigma_0^2} - \frac{1}{2 \sigma_1^2} \Bigr ) (x^*)^2 + \Bigl ( \frac{\mu_1^2}{\sigma_1^2} - \frac{\mu_0^2}{\sigma_0^2} \Bigr ) x^* + \Bigl ( \frac{\mu_0^2}{2 \sigma_0^2} - \frac{\mu_1^2}{2 \sigma_1^2} + \frac{I}{2} \bigl ( \log \sigma_0^2 - \log \sigma_1^2 \bigr ) \Bigr ) &=& 0
\end{eqnarray}

The shape of the decision boundary for the classifier of Section 6.4.1 _does not_ take the form of this equation.  The next problem shows this.

### Problem 6.8



Using the sigmoid function for the conditional probability density function $Pr(w \ | \ x)$, we are looking for coefficients $\phi_0, \phi_1, \phi_2$ that generate the decision boundary given equal priors.

\begin{eqnarray}
\text{sig}[\phi_0 + \phi_1 x^* + \phi_2 (x^*)^2] &=& \frac{1}{2}
\end{eqnarray}

From this it follows that:

\begin{eqnarray}
\exp [-\phi_0 - \phi_1 x^* - \phi_2 (x^*)^2] &=& 1 \\
\text{(take the log)} \ \ -(\phi_0 + \phi_1 x^* + \phi_2 (x^*)^2) &=& 0 \\
\phi_0 + \phi_1 x^* + \phi_2 (x^*)^2 &=& 0
\end{eqnarray}

Following the result of Problem 7 above, we compute the coefficients that give the desired decision boundary:

\begin{eqnarray}
\phi_2 &=& \frac{1}{2 \sigma_0^2} - \frac{1}{2 \sigma_1^2} \\
&\triangleq& \frac{1}{2 \sigma^2} - \frac{1}{2 \cdot 1.5 \sigma^2} \\
&=& \frac{1}{2 \sigma^2} - \frac{1}{3 \sigma^2} = \frac{1}{6 \sigma^2} \\
\phi_1 &=& \frac{\mu_1^2}{\sigma_1^2} - \frac{\mu_0^2}{\sigma_0^2} \\
&=& 0 \\
\phi_0 &=& \frac{\mu_0^2}{2 \sigma_0^2} - \frac{\mu_1^2}{2 \sigma_1^2} + \frac{I}{2} \bigl ( \log \sigma_0^2 + \log \sigma_1^2 \bigr ) &=& 0 - 0 + \frac{I}{2} \bigl ( \log \sigma_0^2 - \log \sigma_1^2 \bigr ) \\
&=& \frac{I}{2} \bigl ( \log \sigma^2 - \log 1.5 \cdot \sigma^2 \bigr ) \\
&=& \frac{I}{2} \bigl ( \log \sigma^2 - \log 1.5 - \log \sigma^2 \bigr ) \\
&=& - \frac{I}{2} \log 1.5 \\
\end{eqnarray}

### Problem 9

For the generative model, the number of parameters to train is $2N$, the number of elements from the two multivariate mean vectors, plus $N(N+1)$, the number of _unique_ elements in the two multivariate covariance matrices, plus 1 for the prior Bernoulli coefficient $\lambda_p$.  The total number of parameters is then $2N + N(N+1) + 1$.

The discriminative model is much simpler.  There are only $N$ weights on the sampled multivariate datapoint $\mathbf{x}^*$ plus 1 for any bias. The total number of parameters is then $N+1$.

For lower dimensions, the training effort for each model will be comparable, and construction of the prior will be straightforward.  The generative model may even provide more insight here, because we could understand an underlying relationship between the world and the data.

For higher dimensions the power of the discriminative classifier dominates.  Constructing any kind of meaningful prior here could be challenging.

### Problem 10

I would set this up as a two-step classification procedure.  First, I would classify pixels into two sets: foreground pixels and background pixels.  Then, given that shadow can sometimes be classified as foreground, I would do a second binary classification on the foreground pixels into _actual_ foreground pixels and shadow pixels.  This second phase would require a retrain on data taken from the original background subtraction step.